<a href="https://colab.research.google.com/github/dmitryglhf/jupyter-projects/blob/main/lab_1_backup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Data

In [ ]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler

In [ ]:
#df = pd.read_csv('cancer_tumor_data_features.csv')

### QR-algorithm


In [ ]:
def householder(a):
    v = a.copy()
    v[0] += np.sign(v[0]) * np.linalg.norm(a)
    H = np.eye(len(a)) - 2 * np.outer(v, v) / np.dot(v, v)  # getting Householder matrix
    return H


def get_qr(A):
    n = A.shape[0]
    Q = np.eye(n)
    R = A.copy()

    for j in range(n):
        a = R[j:, j]  # getting vector
        H = householder(a)  # getting Householder matrix

        R[j:, j:] = H @ R[j:, j:]  # change R
        Q[:, j:] = Q[:, j:] @ H.T  # change Q

    return Q, R


def eig(A, e=1e-10):
    An = A.copy() # for eig values
    eig_vectors = np.eye(A.shape[0]) # for eig vectors
    cache_eig_values = np.diag(An) # diag values
    diff = 1000

    while diff > e:
        Q, R = get_qr(An)

        # update eigen values and eigen vectors
        An = R @ Q
        eig_vectors = eig_vectors @ Q

        # update difference between eig values
        current_eig_values = np.diag(An)
        diff = np.linalg.norm(current_eig_values - cache_eig_values)
        cache_eig_values = current_eig_values

    eig_values = np.diag(An)

    # sort eig_values and eig_vectors
    #sorted_key = np.argsort(eig_values)[::-1]
    #eig_values, eig_vectors = eig_values[sorted_key], eig_vectors[:,sorted_key]

    # return eigen values and eigen vectors
    return eig_values, eig_vectors

### SVD

In [ ]:
def svd(A):
    # V^T matrix
    temp_V = A.T @ A
    eig_values, V = np.linalg.eig(temp_V)  # Собственные значения и векторы для A^T A


    # U matrix
    temp_U = A @ A.T
    _, U = np.linalg.eig(temp_U)  # Собственные значения и векторы для A A^T

    # Sigma matrix
    sigma = np.zeros((A.shape[0], A.shape[1]))
    for i in range(min(A.shape[0], A.shape[1])):
      # тут может быть косяк с плавающей запятой:
      sigma[i, i] = np.sqrt(eig_values[i]) if eig_values[i] >= 0 else 0

    # Result
    return U, sigma, V.T

### Data Scaling

In [ ]:
scaler = StandardScaler()
scaled_X = scaler.fit_transform(df)
scaled_X.mean()

NameError: name 'df' is not defined

### PCA

In [ ]:
def pca(X):
  U, S, V_T = svd(X)

  # finding numer of PC
  sum = S[0][0]
  explained_ratio = sum / np.sum(np.diag(X))
  i = 1
  while(explaned_ratio <= 0.95):
    sum += S[i][i]
    explained_ratio = sum / np.sum(np.diag(X))
    i += 1

  # projection
  V_T_k = V_T[:i, :]  # берем первые k строк матрицы V_T
  X_pca = X @ V_T_k.T

  return X_pca

### Build the Neural Network

https://pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html#model-parameters

In [ ]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

#### Get Device for Training

In [ ]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

NameError: name 'torch' is not defined

#### Define the Class

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

NameError: name 'nn' is not defined

In [ ]:
model = NeuralNetwork().to(device)
print(model)

NameError: name 'NeuralNetwork' is not defined

In [ ]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

### Test - functions

### Test

In [ ]:
  '''
  Для матрицы A_2x2:

  U = [[0.6, 0.8]
        [0.8, -0.6]]

  Sigma = [[3, 0]
                [0, 1]]

  V_T = [[0.8, -0.6]
            [0.6, 0.8]]

  original: http://www.machinelearning.ru/wiki/index.php?title=%D0%A1%D0%B8%D0%BD%D0%B3%D1%83%D0%BB%D1%8F%D1%80%D0%BD%D0%BE%D0%B5_%D1%80%D0%B0%D0%B7%D0%BB%D0%BE%D0%B6%D0%B5%D0%BD%D0%B8%D0%B5
  '''
  A = np.array([[1, 2, 3],
                [5, 7, 8],
                [3, 6, 8]], dtype = float)
svd_test(A)

Origin matrix A: 
[[1. 2. 3.]
 [5. 7. 8.]
 [3. 6. 8.]] 
 
Матрица левых сингулярных векторов U: 
 [[-0.23   0.346 -0.909]
 [-0.727 -0.682 -0.075]
 [-0.646  0.644  0.409]] 
 
Матрица сингулярных чисел Sigma: 
 [[16.097  0.     0.   ]
 [ 0.     1.367  0.   ]
 [ 0.     0.     0.136]] 
 
Матрица правых сингулярных векторов V_T: 
 [[-0.361 -0.586 -0.726]
 [-0.827 -0.158  0.539]
 [ 0.43  -0.795  0.428]] 
 
Исходная матрица: 
 [[0.89325859 2.19710996 2.89392414]
 [4.99116809 7.01630912 7.99122315]
 [3.0480043  5.91135468 8.04770499]] 
 


### Комментарии


#### Заира

Делаем так: нам нужно найти разложение X вида $U*Сигма*V_T$:
- находим собственные значения для матрицы A*A_T и собств вектора -> определим U и Сигму из этого

Cейчас (до вс) нужно научиться сортировать в сигме синг значения по убыванию и менять соответсвующие столбцы в U.
Что делать с матрицей V я подумаю и скажу в вс

#### Дима

__Сделать__

- nenene

#### Вадим

можете оставлять здесь комментарии, заметки и прикладывать картинки.

тыкни два раза по этому тексту чтобы редактировать его

#### Алена

можете оставлять здесь комментарии, заметки и прикладывать картинки.

тыкни два раза по этому тексту чтобы редактировать его

In [ ]:
МЯУ